In [55]:
%matplotlib inline 

import math
import copy
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch import optim, nn

import model
import model_utils
import preprocessing_utils

from os.path import dirname, abspath
import os
import sys

#to be put under model.py when I refactor
from tqdm.notebook import tqdm

In [8]:
torch.manual_seed(1234)

In [35]:
device = torch.device('cpu') # Heroku only have cpu

epochs = 500
batch_size = 32
lr = 0.01
patience = 10

num_timesteps_input = 7 # Default is 30 minutes
num_timesteps_output = 4 # Default is 10 minutes

loss_criterion = nn.MSELoss()

In [10]:
os.getcwd()

'C:\\Users\\daryl\\Desktop\\SUTD school materials\\T7-TheoryAndPracticeOfDeepLearning\\Big Project\\traffic_prediction\\interactive-app'

In [40]:
raw_dir = os.path.join(os.getcwd(), 'data', 'raw')
process_dir = os.path.join(os.getcwd(), 'data', 'processed')

# overwrite = False means that the processing function will only run if the process data files do not exist
# overwrite = True => functions will run regardless
preprocessing_utils.processed(raw_dir, process_dir, overwrite=True)
A, X, metadata, cat2index, means, stds = preprocessing_utils.load(process_dir)


test_original_data = X


test_input, test_target = preprocessing_utils.generate_dataset(test_original_data,
                                           num_timesteps_input=num_timesteps_input,
                                           num_timesteps_output=num_timesteps_output)

# input shape (num_samples ,num_vertices, num_timesteps_window, num_features)
# output shape (num_samples ,num_vertices, num_timesteps_window)
adj_mat = preprocessing_utils.get_normalized_adj(A)
adj_mat = torch.from_numpy(adj_mat).to(device)

Processing C:\Users\daryl\Desktop\SUTD school materials\T7-TheoryAndPracticeOfDeepLearning\Big Project\traffic_prediction\interactive-app\data\raw\Thu_Apr_1_2021\15_00_10.json
Processing C:\Users\daryl\Desktop\SUTD school materials\T7-TheoryAndPracticeOfDeepLearning\Big Project\traffic_prediction\interactive-app\data\raw\Thu_Apr_1_2021\15_05_09.json
Processing C:\Users\daryl\Desktop\SUTD school materials\T7-TheoryAndPracticeOfDeepLearning\Big Project\traffic_prediction\interactive-app\data\raw\Thu_Apr_1_2021\15_10_09.json
Processing C:\Users\daryl\Desktop\SUTD school materials\T7-TheoryAndPracticeOfDeepLearning\Big Project\traffic_prediction\interactive-app\data\raw\Thu_Apr_1_2021\15_15_09.json
Processing C:\Users\daryl\Desktop\SUTD school materials\T7-TheoryAndPracticeOfDeepLearning\Big Project\traffic_prediction\interactive-app\data\raw\Thu_Apr_1_2021\15_20_10.json
Processing C:\Users\daryl\Desktop\SUTD school materials\T7-TheoryAndPracticeOfDeepLearning\Big Project\traffic_predictio

In [58]:
X.shape

(53, 3, 7)

In [59]:
# indices = [(i, i + (num_timesteps_input + num_timesteps_output)) for i
#                in range(X.shape[2] - (
#                 num_timesteps_input + num_timesteps_output) + 1)]
i = 0
indices = [(i, i + (num_timesteps_input + num_timesteps_output))]

# Save samples
features, target = [], []
for i, j in indices:
    features.append(
        X[:, :, i: i + num_timesteps_input].transpose(
            (0, 2, 1)))
    target.append(X[:, 0, i + num_timesteps_input: j])

In [63]:
test_input = torch.from_numpy(np.array(features))

In [64]:
test_target = torch.from_numpy(np.array(target))

In [42]:
# Load model
saved_models_path = os.path.join(dirname(os.getcwd()), 'saved_models', 'last_saved_model.txt')
with open(saved_models_path) as f:
    saved_model = f.read()

latest_model_path = os.path.join(dirname(os.getcwd()), saved_model)
checkpoint = torch.load(latest_model_path, map_location=None)
model_stgcn = model.Stgcn_Model(checkpoint['model_nodes_num'], checkpoint['model_features_num'], checkpoint['model_input_timesteps'], checkpoint['model_num_output'])
model_stgcn.load_state_dict(checkpoint['state_dict'])
optimizer = optim.Adam(model_stgcn.parameters(), lr=checkpoint['model_lr'])
optimizer = optimizer.load_state_dict(checkpoint['opti_state_dict'])

In [43]:
loaded_model = model_stgcn
loaded_optimizer = optimizer

In [65]:
test_input.shape

torch.Size([1, 53, 7, 3])

In [66]:
test_target.shape

torch.Size([1, 53, 0])

In [68]:
torch.manual_seed(1234)


predicted = model_utils.predict(loaded_model, test_input, adj_mat)
predicted_denorm = preprocessing_utils.denormalize(predicted, stds[0], means[0])

In [69]:
predicted

tensor([[[-1.0907, -1.0230, -0.9304, -0.8908],
         [ 0.3436,  0.2000,  0.1298,  0.0950],
         [-0.6031, -0.4171, -0.2409, -0.0788],
         [ 0.3337,  0.2361,  0.1865,  0.1483],
         [ 0.9121,  0.9326,  0.9228,  0.9147],
         [-0.4083, -0.5070, -0.5739, -0.5819],
         [ 0.8408,  0.7398,  0.6884,  0.6627],
         [-0.7209, -0.6660, -0.6274, -0.5984],
         [ 0.7964,  0.8453,  0.8413,  0.8343],
         [-0.6207, -0.6285, -0.6224, -0.6137],
         [ 0.3557,  0.3293,  0.3090,  0.3042],
         [-0.8883, -0.9533, -0.9888, -0.9856],
         [-0.3670, -0.1425,  0.0149,  0.0708],
         [ 0.7834,  0.6583,  0.5853,  0.5092],
         [-0.1691, -0.2074, -0.2067, -0.2188],
         [ 0.3602,  0.3151,  0.2924,  0.2676],
         [-0.2834, -0.3301, -0.3577, -0.3538],
         [-1.8811, -1.7838, -1.6867, -1.6231],
         [-0.5618, -0.3278, -0.1591, -0.0832],
         [ 0.2119,  0.3058,  0.3308,  0.3574],
         [-0.2687, -0.3045, -0.3125, -0.2845],
         [ 0.

In [70]:
predicted_denorm

tensor([[[2.7098, 2.7586, 2.8253, 2.8539],
         [3.7436, 3.6401, 3.5895, 3.5644],
         [3.0612, 3.1953, 3.3223, 3.4391],
         [3.7365, 3.6661, 3.6304, 3.6029],
         [4.1534, 4.1682, 4.1611, 4.1553],
         [3.2017, 3.1305, 3.0823, 3.0765],
         [4.1020, 4.0292, 3.9921, 3.9736],
         [2.9764, 3.0159, 3.0438, 3.0646],
         [4.0700, 4.1052, 4.1023, 4.0973],
         [3.0485, 3.0430, 3.0474, 3.0536],
         [3.7523, 3.7333, 3.7187, 3.7152],
         [2.8557, 2.8088, 2.7832, 2.7856],
         [3.2315, 3.3933, 3.5067, 3.5470],
         [4.0606, 3.9705, 3.9178, 3.8630],
         [3.3741, 3.3465, 3.3470, 3.3382],
         [3.7556, 3.7231, 3.7067, 3.6888],
         [3.2917, 3.2580, 3.2382, 3.2410],
         [2.1401, 2.2103, 2.2802, 2.3261],
         [3.0910, 3.2597, 3.3813, 3.4360],
         [3.6487, 3.7163, 3.7344, 3.7536],
         [3.3023, 3.2764, 3.2707, 3.2909],
         [3.5489, 3.4986, 3.4659, 3.4647],
         [3.0143, 3.0513, 3.0645, 3.0843],
         [3